<a href="https://colab.research.google.com/github/nxxk23/NAKBID/blob/main/NINE/web_scrap_drug_center.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import zipfile
import shutil
import re
import tempfile
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from neo4j import GraphDatabase
from tqdm import tqdm
from neo4j.exceptions import ServiceUnavailable, TransientError
import time
import threading
from fastapi import FastAPI

def login_and_get_download_url(username, password, login_url, download_page):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Run in headless mode
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 10)

    try:
        driver.get(login_url)
        wait.until(EC.presence_of_element_located((By.NAME, "log"))).send_keys(username)
        wait.until(EC.presence_of_element_located((By.NAME, "pwd"))).send_keys(password)
        wait.until(EC.element_to_be_clickable((By.ID, "wp-submit"))).click()
        wait.until(EC.url_changes(login_url))
        print("✅ ล็อกอินสำเร็จ")

        driver.get(download_page)
        download_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a.t-button.t-download")))

        download_url = download_button.get_attribute("href")
        if not download_url:
            raise ValueError("❌ ไม่พบลิงก์ดาวน์โหลด")

        cookies = {cookie['name']: cookie['value'] for cookie in driver.get_cookies()}
        return download_url, cookies
    except Exception as e:
        print(f"❌ เกิดข้อผิดพลาดขณะล็อกอิน: {e}")
        return None, None
    finally:
        driver.quit()

def download_zip_to_temp(download_url, cookies):
    if not download_url:
        raise ValueError("❌ ไม่มี URL สำหรับดาวน์โหลด")

    headers = {"User-Agent": "Mozilla/5.0"}
    session = requests.Session()
    session.cookies.update(cookies)

    response = session.get(download_url, stream=True, headers=headers)
    if response.status_code == 200:
        temp_zip = tempfile.NamedTemporaryFile(delete=False, suffix=".zip")
        with open(temp_zip.name, "wb") as file:
            for chunk in response.iter_content(1024):
                file.write(chunk)
        print(f"✅ ดาวน์โหลดสำเร็จ:")
        return temp_zip.name
    else:
        raise Exception(f"❌ ดาวน์โหลดล้มเหลว (Error {response.status_code})")

def extract_latest_files(zip_path):
    temp_dir = tempfile.mkdtemp()
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(temp_dir)
        print(f"✅ แตกไฟล์ ZIP ไปยัง {temp_dir}")

        # ตรวจสอบไฟล์ที่ถูกแตกออกมา
        extracted_files = os.listdir(temp_dir)
        print("📂 ไฟล์ที่แตกออกมา:", extracted_files)

        return temp_dir
    except zipfile.BadZipFile:
        print("❌ ไฟล์ ZIP เสียหาย")
        return None

def find_date_suffix(base_path):
    print(f"📂 ค้นหาโฟลเดอร์หลักใน: {base_path}")

    # ค้นหาโฟลเดอร์ที่มีวันที่ 8 หลัก
    date_pattern = re.compile(r'TMTRF(\d{8})_BONUS')

    for folder in os.listdir(base_path):
        match = date_pattern.match(folder)
        if match:
            date_suffix = match.group(1)
            full_path = re.sub(r'\\', '/', os.path.normpath(os.path.join(base_path, folder)))
            print(f"📂 พบโฟลเดอร์ข้อมูล: {full_path}")
            print(date_suffix)
            print(full_path)
            return date_suffix, full_path  # คืนค่า date_suffix และ path ของโฟลเดอร์นั้น

    raise ValueError("❌ ไม่พบโฟลเดอร์ TMTRFYYYYMMDD ใน base_path")

def load_excel_files(full_path, date_suffix):
    files = {
        "GP": f"{full_path}/Concept/GP{date_suffix}.xls",
        "VTM": f"{full_path}/Concept/VTM{date_suffix}.xls",
        "SUBS": f"{full_path}/Concept/SUBS{date_suffix}.xls",
        "TP": f"{full_path}/Concept/TP{date_suffix}.xls",
        "TPU": f"{full_path}/Concept/TPU{date_suffix}.xls",
        "GPU": f"{full_path}/Concept/GPU{date_suffix}.xls",
        "tp_tpu": f"{full_path}/Relationship/TPtoTPU{date_suffix}.xls",
        "gp_tp": f"{full_path}/Relationship/GPtoTP{date_suffix}.xls",
        "vtm_gp": f"{full_path}/Relationship/VTMtoGP{date_suffix}.xls",
        "subs_vtm": f"{full_path}/Relationship/SUBStoVTM{date_suffix}.xls",
        "gp_gpu": f"{full_path}/Relationship/GPtoGPU{date_suffix}.xls"
    }

    data = {}

    for key, file_path in files.items():
        if os.path.exists(file_path):
            try:
                data[key] = pd.read_excel(file_path)
                print(f"✅ โหลดไฟล์ {file_path} สำเร็จ")
            except Exception as e:
                print(f"❌ โหลดไฟล์ {file_path} ล้มเหลว: {e}")
        else:
            print(f"⚠️ ไม่พบไฟล์: {file_path}")
    print(data)
    return data

def process_data(data):
    merge_df = data['tp_tpu'].merge(data['gp_tp'], on='TPID', how='left')
    merge_df = merge_df.merge(data['vtm_gp'], on='GPID', how='left')
    merge_df = merge_df.merge(data['subs_vtm'], on='VTMID', how='left')
    merge_df = merge_df.merge(data['gp_gpu'][['GPID', 'GPUID']], on='GPID', how='left')
    merge_df = merge_df.rename(columns={
        'GPID': 'TMTID(GP)', 'TPID': 'TMTID(TP)', 'GPUID': 'TMTID(GPU)', 'TPUID': 'TMTID(TPU)',
        'VTMID': 'TMTID(VTM)', 'SUBSID': 'TMTID(SUBS)'
    })
    print(f"ครั้งที่1 มีแค่ตัวเลข {merge_df.head(2)}")

    merge_df = merge_df.merge(data['GP'][['TMTID(GP)', 'FSN']], on='TMTID(GP)', how='left').rename(columns={'FSN': 'GPNAME'})
    merge_df = merge_df.merge(data['TP'][['TMTID(TP)', 'FSN']], on='TMTID(TP)', how='left').rename(columns={'FSN': 'TPNAME'})
    merge_df = merge_df.merge(data['GPU'][['TMTID(GPU)', 'FSN']], on='TMTID(GPU)', how='left').rename(columns={'FSN': 'GPUNAME'})
    merge_df = merge_df.merge(data['TPU'][['TMTID(TPU)', 'FSN']], on='TMTID(TPU)', how='left').rename(columns={'FSN': 'TPUNAME'})
    merge_df = merge_df.merge(data['VTM'][['TMTID(VTM)', 'FSN']], on='TMTID(VTM)', how='left').rename(columns={'FSN': 'VTMNAME'})
    merge_df = merge_df.merge(data['SUBS'][['TMTID(SUBS)', 'FSN']], on='TMTID(SUBS)', how='left').rename(columns={'FSN': 'SUBSNAME'})
    print(f"ครั้งที่2 มีชื่อ sub {merge_df.head(2)}")

    merge_df = merge_df.groupby(['TMTID(TPU)', 'TPUNAME','TMTID(TP)', 'TPNAME', 'TMTID(GPU)', 'GPUNAME',
                                   'TMTID(GP)', 'GPNAME','TMTID(VTM)','VTMNAME'], as_index=False).agg({
        'TMTID(SUBS)': list, 'SUBSNAME': list
    })
    merge_df = merge_df.rename(columns={'TMTID(SUBS)': 'SUBSID_LIST', 'SUBSNAME': 'SUBS_LIST'})
    print(f"ครั้งที่3 มี sublist {merge_df.head(2)}")
    return merge_df

def update_neo4j(df):
    uri = "neo4j+s://cc0a9592.databases.neo4j.io"
    user = "neo4j"
    password = "Zu6N2LzjUJ9o_a1ycDMi8WNON4wW3hkS3p-OMMqdONw"
    driver = GraphDatabase.driver(uri, auth=(user, password))

    retries = 0
    MAX_RETRIES_NEO = 3
    RETRY_DELAY_NEO = 60  # วินาที

    while retries < MAX_RETRIES_NEO:
        try:
            with driver.session() as session:
                for _, row in tqdm(df.iterrows(), total=df.shape[0], desc="Updating Neo4j"):
                    session.run("""
                        MERGE (n:DRUG { `TMTID(GP)`: $gp_id })
                        SET n.`TMTID(TPU)` = $tpu_id,
                            n.`TPUNAME` = $tpu_name,
                            n.`TMTID(TP)` = $tp_id,
                            n.`TPNAME` = $tp_name,
                            n.`TMTID(GPU)` = $gpu_id,
                            n.`GPUNAME` = $gpu_name,
                            n.`GPNAME` = $gp_name,
                            n.`TMTID(VTM)` = $vtm_id,
                            n.`VTMNAME` = $vtm_name,
                            n.`SUBSID_LIST` = $subs_list,
                            n.`SUBS_LIST` = $subs_names
                    """,
                    gp_id=row['TMTID(GP)'], tpu_id=row['TMTID(TPU)'], tpu_name=row['TPUNAME'],
                    tp_id=row['TMTID(TP)'], tp_name=row['TPNAME'], gpu_id=row['TMTID(GPU)'],
                    gpu_name=row['GPUNAME'], gp_name=row['GPNAME'], vtm_id=row['TMTID(VTM)'],
                    vtm_name=row['VTMNAME'], subs_list=row['SUBSID_LIST'], subs_names=row['SUBS_LIST'])

            print("✅ อัปเดต Neo4j สำเร็จ")
            return  # ออกจาก loop ถ้าอัปเดตสำเร็จ

        except ServiceUnavailable as e:
            print(f"⚠️ Neo4j ไม่พร้อมใช้งาน Sever ไม่ตอบสนอง: {e}")
        except TransientError as e:
            print(f"🔄 พบข้อผิดพลาดชั่วคราว: {e} → กำลังลองใหม่...")
        except Exception as e:
            print(f"❌ ข้อผิดพลาดทั่วไปใน Neo4j: {e}")
            break  # หยุดทำงานทันทีถ้าเป็น error ที่ไม่สามารถ retry ได้

        retries += 1
        if retries < MAX_RETRIES_NEO:
            print(f"🔁 ลองใหม่ใน {RETRY_DELAY_NEO} วินาที...")
            time.sleep(RETRY_DELAY_NEO)
        else:
            print("❌ ล้มเหลวทุกครั้ง → หยุดทำงาน")

    driver.close()



MAX_RETRIES = 5  # จำนวนครั้งสูงสุดที่ลองใหม่
RETRY_DELAY = 60  # เวลาหน่วงระหว่าง retry (วินาที)

def main():
    username = "chanatip0615@gmail.com"
    password = "#chanatip2011"
    login_url = "https://www.this.or.th/en/account/"
    download_page = "https://www.this.or.th/en/download/"

    retries = 0
    while retries < MAX_RETRIES:
        try:
            print(f"🚀 กำลังรันสคริปต์... (ลองครั้งที่ {retries + 1})")

            download_url, cookies = login_and_get_download_url(username, password, login_url, download_page)
            if not download_url:
                raise Exception("❌ ไม่พบ URL สำหรับดาวน์โหลด")

            temp_zip_path = download_zip_to_temp(download_url, cookies)
            temp_extract_dir = extract_latest_files(temp_zip_path)

            if not temp_extract_dir:
                raise Exception("❌ ไม่สามารถแตกไฟล์ zip ได้")

            date_suffix, full_path = find_date_suffix(temp_extract_dir)
            data = load_excel_files(full_path, date_suffix)
            print("📊 โหลดข้อมูลสำเร็จ:", data.keys())

            if not data:
                raise Exception("❌ ไม่พบข้อมูลในไฟล์ Excel")

            processed_data = process_data(data)
            update_neo4j(processed_data)

            print("✅ รันสำเร็จ!")
            return  # จบการทำงานถ้าสำเร็จ

        except Exception as e:
            print(f"⚠️ พบข้อผิดพลาด: {e}")
            retries += 1
            if retries < MAX_RETRIES:
                print(f"🔄 ลองใหม่ใน {RETRY_DELAY} วินาที...")
                time.sleep(RETRY_DELAY)
            else:
                print("❌ ล้มเหลวทุกครั้ง → หยุดทำงาน")
                return

        finally:
            if 'temp_zip_path' in locals() and os.path.exists(temp_zip_path):
                os.remove(temp_zip_path)
                print("🗑️ ลบไฟล์ ZIP สำเร็จ")

            if 'temp_extract_dir' in locals() and os.path.exists(temp_extract_dir):
                shutil.rmtree(temp_extract_dir)
                print("🗑️ ลบโฟลเดอร์ชั่วคราวสำเร็จ")

if __name__ == "__main__":
    main()


# # สร้าง FastAPI
# app = FastAPI()

# # สร้าง API Endpoint
# @app.get("/")
# def read_root():
#     return {"message": "🚀 API พร้อมใช้งาน! เรียก /run เพื่อรันสคริปต์"}

# @app.get("/run")
# def run():
#     thread = threading.Thread(target=main)
#     thread.start()
#     return {"status": "running", "message": "🔄 สคริปต์กำลังทำงาน กรุณาตรวจสอบ Console"}

# # รัน FastAPI ด้วย Uvicorn
# if __name__ == "__main__":
#     import uvicorn
#     uvicorn.run(app, host="0.0.0.0", port=8000)